In [ ]:
!pip install transformers -q
!pip install streamlit -q
!pip install anthropic -q
!pip install pypdf -q
!pip install pyngrok -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.9/891.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.8/292.8 kB 7.2 MB/s eta 0:00:00


In [ ]:
%%writefile app.py
import streamlit as st
import requests, os
from anthropic import Anthropic
from pypdf import PdfReader

client = Anthropic(
    api_key=''
)
MODEL_NAME = 'claude-3-5-sonnet-20240620'

# Intro Text
st.markdown('# PDF summarization using Generative AI')
st.header('''PDF size must be below 200MB. Only 1 PDF per time, time will vary depending on size''')

input_file = st.file_uploader('Upload a PDF file')

# Save uploaded file
if input_file:
  with open(os.path.join(input_file.name),'wb') as f:
        f.write(input_file.getbuffer())
  st.success('Saved File')

st.markdown('# Summary')

def read_pdf(pdf_file):
  reader = PdfReader(pdf_file)
  content = ''.join(page.extract_text() for page in reader.pages)
  return content

def get_completion(_client, _prompt):
  return _client.messages.create(
      model=MODEL_NAME,
      max_tokens=2048,
      messages=[{
          "role": 'user', "content":  _prompt
      }]
  ).content[0].text

test = st.text_input("Question", "")

if input_file and test:

  text = read_pdf(input_file)
  completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

  Please do the following:
  Answer all questions asked about the aademic paper. The question is located here: <question>{test}</question>
  """
  )
  st.success('Summary ready!')
  st.text_area(label = '', value = completion, placeholder = 'Please upload a PDF to summarise', height = 400)
  st.markdown('')

else:
  st.text_area(label = '',value = 'Please upload a PDF to summarise', placeholder = 'Please upload a PDF to summarise', height = 400)

Overwriting app.py


In [ ]:
from pyngrok import ngrok

# Auth Token
ngrok.set_auth_token('')

# Start Streamlit server and set port number
!nohup streamlit run app.py --server.port 5011 &

# Start ngrok tunnel
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)
print('Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
Tunnel URL: https://e08e-34-16-172-146.ngrok-free.app


In [ ]:
# Shut down tunnel
print(ngrok.get_tunnels())
ngrok.disconnect(ngrok_tunnel.public_url)
print(ngrok.get_tunnels())